# side model feature selection

In [1]:
import pandas as pd

df_features = pd.read_parquet("data/features_4h.parquet")
print(df_features.shape)
df_features.isna().sum(axis=0).sort_values(ascending=False)

(11332, 11841)


16h_dft_dom_cycle_ddt_lag39    361
16h_dft_dom_cycle_ddt_lag38    357
16h_dft_dom_cycle_dt_lag39     357
16h_dft_dom_cycle_lag39        353
16h_dft_dom_cycle_ddt_lag37    353
                              ... 
4h_comb_spectrum_pwr_28          0
4h_comb_spectrum_pwr_27          0
4h_comb_spectrum_pwr_26          0
4h_comb_spectrum_pwr_24          0
4h_ac_42                         0
Length: 11841, dtype: int64

In [2]:
import numpy as np
from jesse import utils

label = np.load("data/label_side.npy")
df_label = utils.numpy_candles_to_dataframe(label[:, :6]).set_index("date")
df_label["side_label"] = label[:, 6]
df_label.head(1)

,open,close,high,low,volume,side_label
date,,,,,,
2020-01-01 15:52:00,7222.68,7340.97,7407.28,6863.44,294315.467,1.0


In [3]:
side_features = df_features.iloc[361:]
side_label = df_label.iloc[361:]

side_label["side_label"] = (side_label["side_label"].astype(int) == 1).astype(int)
side_label = side_label["side_label"].to_numpy()

print(side_features.shape)
print(side_label.shape)

side_features.isna().sum(axis=0).sort_values(ascending=False)

(10971, 11841)
(10971,)


/var/folders/2g/md1t2j7s6hjdjc5jlvkmn7_c0000gn/T/ipykernel_60817/2679961447.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_label["side_label"] = (side_label["side_label"].astype(int) == 1).astype(int)


16h_ac_0                   0
1h_ac_4                    0
4h_williams_r_ddt_lag35    0
4h_williams_r_ddt_lag36    0
4h_williams_r_ddt_lag37    0
                          ..
4h_ac_2                    0
4h_ac_3                    0
4h_ac_4                    0
4h_ac_5                    0
1h_williams_r_ddt_lag39    0
Length: 11841, dtype: int64

In [4]:
from custom_indicators.toolbox.feature_selection.fcq_selector import FCQSelector

selector = FCQSelector(max_features=3000)
selector.fit(side_features, side_label)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性...
✓ 选择第1个特征: 16h_williams_r (最大F值: 3724.8525)
➤ 计算特征冗余度...
➤ 总计选择3000个特征 (已选择1个，还需选择2999个)...
➤ 开始MRMR迭代选择过程...


选择特征: 100%|███████████████████████████████████████████████| 2999/2999 [08:06<00:00,  6.16特征/s]



✅ 特征选择完成：从11841个特征中选择了3000个，舍弃了8841个
✅ 选择的特征: ['16h_williams_r', '16h_acc_swing_index_dt_lag37', '16h_evenbetter_sinewave_short_lag11', '4h_acc_swing_index_dt', '4h_trendflex', '16h_comb_spectrum_pwr_8', '4h_williams_r', '16h_williams_r_lag1', '16h_hurst_coef_slow_dt_lag15', '4h_williams_r_lag1', '16h_trendflex', '4h_fisher', '4h_trendflex_lag1', '16h_evenbetter_sinewave_long', '4h_evenbetter_sinewave_long', '16h_fisher', '4h_williams_r_lag2', '16h_acp_pwr_1', '4h_fisher_lag1', '16h_comb_spectrum_pwr_9', '16h_evenbetter_sinewave_short', '4h_trendflex_lag2', '1h_williams_r', '4h_evenbetter_sinewave_long_lag1', '16h_trendflex_lag1', '1h_trendflex', '4h_evenbetter_sinewave_short', '16h_williams_r_lag2', '4h_trendflex_lag3', '16h_acc_swing_index_dt', '16h_comb_spectrum_pwr_10', '16h_hurst_coef_slow_dt_lag7', '4h_fisher_lag2', '16h_trendflex_lag2', '4h_vwap_dt', '4h_evenbetter_sinewave_long_lag2', '16h_ac_46', '4h_williams_r_lag3', '16h_evenbetter_sinewave_long_lag1', '1h_fisher', '16h_com

In [5]:
side_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)
side_res

16h_williams_r                          3.724853e+03
4h_williams_r                           2.696688e+03
4h_trendflex                            2.660053e+03
4h_fisher                               2.237988e+03
4h_trendflex_lag1                       2.224145e+03
                                            ...     
4h_dft_dom_cycle_dt_lag33               1.290740e-07
4h_comb_spectrum_dom_cycle_ddt_lag12    6.069964e-08
4h_stc_dt_lag30                         1.100904e-08
4h_pfe_ddt_lag31                        2.282037e-09
1h_mod_rsi_dt_lag23                     2.272020e-10
Length: 11841, dtype: float64

In [6]:
import json

with open(f"data/side_features_{selector.__class__.__name__}.json", "w") as f:
    json.dump(side_res.to_dict(), f, indent=4)

In [7]:
import json

import pandas as pd

with open(f"data/side_features_FCQSelector.json", "r") as f:
    side_res = pd.Series(json.load(f))

In [8]:
DOLLAR_BAR_SHORT_TERM = "1h"
DOLLAR_BAR_MEDIUM_TERM = "4h"
DOLLAR_BAR_LONG_TERM = "16h"

feature_info = {
    "side": {
        DOLLAR_BAR_SHORT_TERM: [],
        DOLLAR_BAR_MEDIUM_TERM: [],
        DOLLAR_BAR_LONG_TERM: [],
    },
    "meta": {
        DOLLAR_BAR_SHORT_TERM: [],
        DOLLAR_BAR_MEDIUM_TERM: [],
        DOLLAR_BAR_LONG_TERM: [],
        "model_res": [],
    },
}

for k, v in side_res[:3000].to_dict().items():
    if k.startswith(f"{DOLLAR_BAR_SHORT_TERM}_"):
        feature_info["side"][DOLLAR_BAR_SHORT_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_MEDIUM_TERM}_"):
        feature_info["side"][DOLLAR_BAR_MEDIUM_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_LONG_TERM}_"):
        feature_info["side"][DOLLAR_BAR_LONG_TERM].append(k)

feature_info["side"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["side"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["side"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_LONG_TERM])),
)

feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_LONG_TERM])),
)
feature_info["meta"]["model_res"] = sorted(
    list(set(feature_info["meta"]["model_res"])),
)

with open("model/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)

# meta model feature selection

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from model.config import SIDE_ALL

df_features = pd.read_parquet("data/features_4h.parquet")
meta_label = np.load("data/label_meta.npy")

side_model = lgb.Booster(model_file="model/model_side.txt")
side_model_pred_prob = side_model.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_pred_prob

assert df_features.shape[0] == meta_label.shape[0]
assert "side_model_res" in df_features.columns

df_features.isna().sum(axis=0).sort_values(ascending=False)

16h_dft_dom_cycle_ddt_lag39    361
16h_dft_dom_cycle_ddt_lag38    357
16h_dft_dom_cycle_dt_lag39     357
16h_dft_dom_cycle_lag39        353
16h_dft_dom_cycle_ddt_lag37    353
                              ... 
4h_comb_spectrum_pwr_14          0
4h_comb_spectrum_pwr_13          0
4h_comb_spectrum_pwr_12          0
4h_comb_spectrum_pwr_11          0
side_model_res                   0
Length: 11842, dtype: int64

In [2]:
meta_features = df_features[361:]
meta_label = meta_label[361:]

meta_label = pd.Series(meta_label[:, 6].astype(int), index=meta_features.index)

# meta_features = meta_features[meta_label["ret"].notna()]
# meta_label = meta_label[meta_label["ret"].notna()]["bin"]

print(meta_features.shape)
print(meta_label.shape)

meta_features.isna().sum(axis=0).sort_values(ascending=False)

(10971, 11842)
(10971,)


16h_ac_0                   0
1h_ac_4                    0
4h_williams_r_ddt_lag35    0
4h_williams_r_ddt_lag36    0
4h_williams_r_ddt_lag37    0
                          ..
4h_ac_2                    0
4h_ac_3                    0
4h_ac_4                    0
4h_ac_5                    0
side_model_res             0
Length: 11842, dtype: int64

In [3]:
# meta feature selection
from custom_indicators.toolbox.feature_selection.fcq_selector import FCQSelector

selector = FCQSelector(max_features=3000)
selector.fit(meta_features, meta_label)
meta_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性...
✓ 选择第1个特征: 16h_vwap_lag29 (最大F值: 77.4005)
➤ 计算特征冗余度...
➤ 总计选择3000个特征 (已选择1个，还需选择2999个)...
➤ 开始MRMR迭代选择过程...


选择特征: 100%|███████████████████████████████████████████████| 2999/2999 [08:02<00:00,  6.22特征/s]



✅ 特征选择完成：从11842个特征中选择了3000个，舍弃了8842个
✅ 选择的特征: ['16h_vwap_lag29', '4h_ehlers_early_onset_trend_ddt_lag3', '16h_mod_rsi_dt_lag32', '4h_natr_ddt', '16h_comb_spectrum_pwr_11', '4h_adaptive_rsi_dt_lag37', '16h_hurst_coef_fast_ddt_lag31', '4h_adaptive_rsi_dt_lag14', '16h_hurst_coef_slow_ddt_lag36', '1h_comb_spectrum_dom_cycle_dt_lag39', '4h_hurst_coef_slow_ddt_lag15', '16h_adaptive_bp_lead_lag18', '4h_forecast_oscillator_lag22', '16h_acp_pwr_7', '16h_comb_spectrum_dom_cycle_dt_lag2', '16h_ac_29', '1h_adaptive_bp_lead_ddt_lag8', '4h_ac_21', '16h_phase_accumulation_dt_lag12', '16h_adaptive_bp_dt_lag39', '1h_evenbetter_sinewave_short_ddt_lag38', '16h_ehlers_early_onset_trend_ddt_lag3', '1h_natr_dt_lag25', '4h_acp_pwr_1', '4h_dual_diff_ddt_lag27', '16h_vwap_lag28', '16h_acp_pwr_4', '16h_williams_r_ddt_lag37', '16h_hurst_coef_fast_dt_lag32', '4h_dual_diff_lag5', '4h_pfe_ddt_lag28', '16h_homodyne_ddt_lag19', '4h_evenbetter_sinewave_short_lag21', '16h_comb_spectrum_pwr_9', '16h_ehlers_early_onset_

In [4]:
import json

with open(f"data/meta_features_{selector.__class__.__name__}.json", "w") as f:
    json.dump(meta_res.to_dict(), f, indent=4)

In [5]:
import json

import pandas as pd

with open("data/meta_features_FCQSelector.json", "r") as f:
    meta_res = pd.Series(json.load(f))

In [6]:
meta_res.to_dict()["side_model_res"]

32.17089213990211

In [7]:
from pathlib import Path

feature_info = Path("model/feature_info.json")
if feature_info.exists():
    with open(feature_info, "r") as f:
        feature_info = json.load(f)

DOLLAR_BAR_SHORT_TERM = "1h"
DOLLAR_BAR_MEDIUM_TERM = "4h"
DOLLAR_BAR_LONG_TERM = "16h"


feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = []
feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM] = []
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = []
feature_info["meta"]["model_res"] = []

for k, v in meta_res[:3000].to_dict().items():
    if k.startswith(f"{DOLLAR_BAR_SHORT_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_SHORT_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_MEDIUM_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_LONG_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_LONG_TERM].append(k)
    else:
        feature_info["meta"]["model_res"].append(k)

feature_info["side"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["side"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["side"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_LONG_TERM])),
)

feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_LONG_TERM])),
)
feature_info["meta"]["model_res"] = sorted(
    list(set(feature_info["meta"]["model_res"])),
)

with open("model/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)